# Ansible Scripting Tutorial
<i>Adapted for use with FABRIC from [Hello Ansible](https://groups.geni.net/geni/wiki/GENIExperimenter/Tutorials/AnsibleHelloGENI)

<b> Prerequisites  
    
* You need to have your FABRIC bastion host key pair set up to do this tutorial. If you have not already set this up, follow steps 1-3 at https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/.
* You are comfortable using ssh and executing basic commands using a UNIX shell. [Tips about how to login to hosts.](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/)
  
This is the second step in this assignment, to go to the previous step go to slice creation notebook or click [Here](./CreateSlice.ipynb)
<br><img src="./Figures/AnsinbleTop.png">


## 1. Retrieve Slice
Create the slice at the [Create Slice Notebook](./CreateSlice.ipynb) and import it here.


#### Import the Fabric API

In [ ]:
# Load Fablib and Node Information
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()                    
fablib.show_config()
import json
import traceback

slice_name = 'ExploringAnsible'
try:
    slice = fablib.get_slice(slice_name)   
    slice.list_nodes()
except Exception as e:
    print(f"Get Slices Fail: {e}")

## 2. Guieded Experiment

### 2.1 Configure and initialize
To start ansible you will have to follow some steps to get the connection set up
1. Create your Ansible inventory file: 
<br> on your Ansible node open a Terminal and run the following command to see the available nodes:
<br> ```$ cat inventory```
2. Be sure your private key has been added to your SSH agent:
<br> ```ssh-agent bash```
<br> ```ssh-add ~/Ansible_Key```
3. Check to see if your nodes are up and ready. 
<br> ```$ ansible -i inventory all -m ping ```
<br> the output of the command should look like this:
<br> ```   server | SUCCESS => {                                      |```
<br> ```    "ansible_facts":                                          |```
<br> ```        "discovered_interpreter_python": "/usr/bin/python3"   |```
<br> ```    },                                                        |```
<br> ```    "changed": false,                                         |```
<br> ```    "ping": "pong"                                            |```
<br> ``` }                                                            |```
<br> ``` client | SUCCESS => {                                        |```
<br> ```     "ansible_facts": {                                       |```
<br> ```         "discovered_interpreter_python": "/usr/bin/python3"  |```
<br> ```     },                                                       |```
<br> ```    "changed": false,                                         |```
<br> ```   "ping": "pong"                                             |```
<br> ``` }                                                            |```
4. Try using the ping module in Ansible to only ping server or client by replacing all in the above with server or client. 

### 2.2 Introduction to ansible
#### 2.2.1 Ansible Commands
The following are some example Ansible Ad Hoc commands. You can run these commands one at a time from the machine where you have Ansible installed. `-b` tells Ansible to use `sudo` when executing the command. All modules are described in the [Ansible Documentation](https://docs.ansible.com/ansible/latest/module_plugin_guide/index.html). 

<br> [**apt**](https://docs.ansible.com/ansible/2.3/apt_module.html) module is used to install packages using the apt package manager: 
<br>`  ansible [-i inventory] [all/server/client] -b -m apt -a "name=apache2 update_cache=yes"`
<br> [**command**](https://docs.ansible.com/ansible/2.3/command_module.html) module is used to execute an arbitrary command on the remote node: 
<br>`  ansible [-i inventory] [all/server/client]  -b -m command -a "/usr/sbin/a2enmod status"`
<br> [**file**](https://docs.ansible.com/ansible/2.3/file_module.html) module is used to set attributes of files:
<br>`  ansible [-i inventory] [all/server/client] -b -m file -a "path=/var/www/html state=absent"`
<br> [**synchronize**](https://docs.ansible.com/ansible/2.4/synchronize_module.html) module is an implementation of rsync and is used to efficiently synchronize files between your local machine and a remote node:
<br>`  ansible [-i inventory] [all/server/client] -b  -m synchronize \ -a "src=website/index.html dest=/var/www"`
<br> [**lineinfile**](https://docs.ansible.com/ansible/latest/collections/ansible/builtin/lineinfile_module.html) module is used to see if an arbitrary line exists in a file: 
<br>`  ansible [-i inventory] [all/server/client] -b  -m lineinfile \
     -a "line='ExtendedStatus On' dest=/etc/apache2/conf.d/extendedstatus create=yes state=present"`
<br> [**service**](https://docs.ansible.com/ansible/2.9_ja/modules/service_module.html) module is used to start/stop/restart/etc services: 
<br>`  ansible [-i inventory] [all/server/client] -b  -m service -a "name=apache2 state=restarted"`


#### 2.2.1 Ansible Playbook
<br>Ansible commands can be collected into files called Playbooks. Playbooks are in a configuration file format called YAML which is very straightforward. In particular, Ansible Ad Hoc commands easily map to commands used in an Ansible Playbook.

The Playbook to configure the client node as in the install script is as follows:
<br>```   - name: Configure client                                   | ```
<br>```     hosts: client                                            | ```
<br>```     tasks:                                                   | ```
<br>```    - name: install apache2                                   | ```
<br>```      apt: name=apache2 update_cache=yes                      | ```
<br>```      become: yes                                             | ```
<br>```    - name: install iperf                                     | ```
<br>```      apt: name=iperf update_cache=yes                        | ```
<br>```      become: yes                                             | ```
<br>```    - name: copy scripts into /local with permissions 755     | ```
<br>```        synchronize: src=scripts dest=/local mode=755         | ```

Do these commands look like the Ad Hoc commands you came up with in the previous step?

Put the above content in a file called ```hello-client.yml```

Run the playbook with the following command on the local machine: 
<br>```ansible-playbook hello-client.yml -i inventory```



### 2.3 Exercises

#### 2.3.1 E1 Ansible Commands:
Try using the above Ad Hoc Ansible commands in section 2.2.1 to configure the client node by doing the following:
1. On both client and server, install ```apache2``` and ```iperf```
2. Copy the ```scripts directory``` into ```/local``` on the client node with permissions ```755```.
3. Verify the above three items have been done
4. Log into the client node and run:
<br>```   $ apache2 -v```
<br>```   $ iperf -v   ```
<br>```   $ ls /local/scripts```


#### 2.3.1 E2 Ansible Playbook:
Using the above Ansible modules, configure the ```server``` node by reproducing the steps in the HelloGENI install script for the server node. As you find a command that works, use it to construct a ```hello-server.yml``` playbook. (See the yml [solution](https://github.com/GENI-NSF/geni-tutorials/blob/master/AnsibleHelloGENI/hello-server.yml) or download the raw [hello-server.yml](https://raw.githubusercontent.com/GENI-NSF/geni-tutorials/master/AnsibleHelloGENI/hello-server.yml))

<br> ```- name: Configure server                                                                    |```
<br> ```  hosts: server                                                                             |```
<br> ```  tasks:                                                                                    |```
<br> ```   - name: install apache2                                                                  |```
<br> ```     apt: name=apache2 update_cache=yes                                                     |```
<br> ```     become: yes                                                                            |```
<br> ```   - name: install iperf                                                                    |```
<br> ```     apt: name=iperf update_cache=yes                                                       |```
<br> ```     become: yes                                                                            |```
<br> ```   - name: /usr/sbin/a2enmod status                                                         |```
<br> ```     # INSERT COMMAND HERE                                                                  |```
<br> ```   - name: check /etc/apache2/mods-enabled/status.conf file is absent                       |```
<br> ```     # INSERT COMMAND HERE                                                                  |```
<br> ```   - name: copy website/index.html into /var/www                                            |```
<br> ```     # INSERT COMMAND HERE                                                                  |```
<br> ```   - name: copy the website/graphics directory into /var/www                                |```
<br> ```     # INSERT COMMAND HERE                                                                  |```
<br> ```   - name: rm -rf /var/www/html                                                             |```
<br> ```     # INSERT COMMAND HERE                                                                  |```
<br> ```   - name: Make simlink to webfiles                                                         |```
<br> ```     # INSERT COMMAND HERE                                                                  |```
<br> ```   - name: restart apache2 service                                                          |```
<br> ```     # INSERT COMMAND HERE                                                                  |```
<br> ```   - name: Make sure /etc/apache2/conf.d/extendedstatus file contains "ExtendedStatus On"   |```
<br> ```     # INSERT COMMAND HER                                                                   |```
<br> ```   - name: Make sure /etc/apache2/sites-available/default contains Location information     |```
<br> ```     # INSERT COMMAND HERE                                                                  |```
<br> ```   - name: create directory for iperf logs in /var/www/iperflogs with permissions of 755    |```
<br> ```     # INSERT COMMAND HERE                                                                  |```

1. Run both the client and server playbooks on your nodes.
2. Start the iperf and wget traffic. **STOP HERE and SPEAK TO THE INSTRUCTOR**.
    - On the server node do:
    <br> ```iperf_server_log="/var/www/iperflogs/iperf-server.log"```
    <br> ```sudo bash -c "iperf -s -i 10 &> $iperf_server_log"```
    - On the client node do:
    <br> ```  /local/scripts/client-wget.sh&```
    <br> ```  /local/scripts/client-wget.sh&```

## 3. Assignment
### Part 1
1. You may have noticed that the server playbook and the client playbook contained duplicate commands. This is because both nodes need to have apache installed so that they can act as webservers. As you might have guessed, there is a way to modularize this configuration. In Ansible this is done with "roles".
2. Explore some of the intermediate features of Ansible such as roles, variables, and handlers.

### Part 2
1. Reserve resource in a second slice ansible2XX (where XX is your initials), and test your playbooks from scratch in the new slice. Do you get the same behavior?

If this was a real experiment, you would commit your Ansible playbook to a version control system like git as you go.

## Cleanup Resources

Once you have completed the assignment shut down the slice


In [ ]:
# Delete Slice
try:
    #To delete the slice change "CHECK" to "True", this is to prevent accidental slice deletion
    CHECK = False
    if (CHECK):
        slice = fablib.get_slice(slice_name)
        slice.delete()
    else:
        print("Change the Boolean to delete slice")
except Exception as e:
    print(f"Fail: {e}")